In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open("../pickle_files/user_user_training_matrix_2000.pickle", "rb") as f:
    train_data = pickle.load(f)

with open("../pickle_files/cos_result_sum_2000.pickle", "rb") as f:
    train_label = pickle.load(f)

print(type(train_data))
print(type(train_label))

<class 'dict'>
<class 'numpy.ndarray'>


In [3]:
columns = ('education', 'religion', 'bodyType', 'diet', 'age', 'smokes', 'drinks', 'pets')
index = pd.MultiIndex.from_tuples(train_data.keys())
df = pd.DataFrame.from_dict(train_data, orient='index', columns=columns)
df.head(5)

,education,religion,bodyType,diet,age,smokes,drinks,pets
"(8195, 8195)",1.0,1.0,0.869592,1.0,1.00,1.0,1.0,2
"(8195, 8202)",0.5,0.5,0.934448,1.0,0.00,0.0,0.0,2
"(8195, 13)",0.5,0.5,0.787254,1.0,0.25,1.0,1.0,0
"(8195, 8207)",1.0,0.5,0.869592,1.0,0.25,1.0,1.0,0
"(8195, 17)",1.0,0.5,0.869592,1.0,0.50,1.0,1.0,2


In [4]:
data_label = pd.DataFrame(train_label)

In [5]:
print("label shape:",data_label.shape)
data_label.head(5)

label shape: (2000, 2000)


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,10.000001,3.258006,2.172685,3.288786,4.524014,4.262863,4.425867,1.952833,3.780606,3.277067,...,4.393709,5.183300,4.408846,4.114955,5.644509,2.207893,2.712091,2.719143,2.005965,4.570388
1,3.258006,10.000000,3.581994,3.532227,3.974113,4.040970,4.818295,2.221856,3.081551,3.153311,...,4.663496,4.541522,4.414474,4.727452,2.826968,2.756586,3.080917,2.130292,3.932566,3.323575
2,2.172685,3.581994,10.000001,4.211419,3.140306,3.088926,3.565549,3.222371,3.639046,1.594120,...,3.612458,3.501130,3.931247,3.628038,2.791378,3.634598,5.535552,2.814210,4.580935,3.860979
3,3.288786,3.532227,4.211419,10.000001,3.013701,3.390862,2.567312,5.899020,3.066927,4.088910,...,2.621695,2.422608,3.039518,3.297323,3.078377,4.652130,3.313797,2.959043,6.235013,4.097363
4,4.524014,3.974113,3.140306,3.013701,10.000001,4.937578,5.013460,0.762407,3.319923,3.190665,...,4.512901,5.185453,4.652572,4.604312,4.106547,2.418272,3.866645,2.982009,2.751265,4.199334


In [6]:
user_index = []
for i in tqdm(index):
    if i[0] not in user_index:
        user_index.append(i[0])

100%|█████████████████████████████████████████████████████████████████████| 4000000/4000000 [00:59<00:00, 67492.87it/s]


In [7]:
data_label.index = user_index
data_label.columns = user_index

In [11]:
np_data_label = data_label.to_numpy()

In [12]:
np_data_label.shape

(2000, 2000)

In [20]:
data_label_flatten = np_data_label.flatten('C')

In [108]:
data_label_df.head(5)

,cos_sim
"(8195, 8195)",10.000001
"(8195, 8202)",3.258006
"(8195, 13)",2.172685
"(8195, 8207)",3.288786
"(8195, 17)",4.524014


In [27]:
data_label_df = pd.DataFrame(data_label_flatten, index=df.index, columns=['cos_sim'])

# Model training

In [42]:
from sklearn import linear_model, svm
from sklearn.model_selection import GridSearchCV
import ast
import random

In [30]:
print("Running Linear Regression")
model = linear_model.LinearRegression()

Running Linear Regression


In [31]:
parameters={}
grid = GridSearchCV(model,parameters, cv=None)
grid.fit(df, data_label_df)

GridSearchCV(estimator=LinearRegression(), param_grid={})

In [33]:
#dump classifier to a file 
with open('logistic_regrssion.log', 'wb') as fid:
    pickle.dump(grid, fid)  

# Model Validation

# validation set 불러와서 추천 top 5 목록 출력하기

In [56]:
with open('logistic_regression.log', 'rb') as fid:
        logistic_classifier = pickle.load(fid)

In [67]:
len(df.iloc[0])

8

In [83]:
df.shape

(4000000, 8)

In [84]:
feature_vector = df

In [89]:
feature_vector = np.array(feature_vector)

In [92]:
result= logistic_classifier.predict(feature_vector)

In [95]:
result.shape

(4000000, 1)

In [96]:
result

array([[3.86608749],
       [3.72619792],
       [3.74493968],
       ...,
       [3.73982067],
       [3.73518154],
       [3.85661694]])

In [107]:
result[result < 3.9]

array([3.86608749, 3.72619792, 3.74493968, ..., 3.73982067, 3.73518154,
       3.85661694])

In [109]:
feature_label = np.array(data_label_df)

In [110]:
feature_label.shape

(4000000, 1)

In [113]:
cost = result - feature_label

In [114]:
cost.sum()

1.178705133497715e-08